In [1]:
from pathlib import Path
import re
import requests
import time
import datetime

import pandas as pd
from requests_html import HTML
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / "data" # os.path.join(BASE_DIR, 'data')
if not DATA_DIR.exists(): # os.path.exists(DATA_DIR)
    DATA_DIR.mkdir(exist_ok=True) # os.makedirs(DATA_DIR, exist_ok=True)
    
product_category_links_output = DATA_DIR / "category-products.csv"
product_output = DATA_DIR / "products.csv"

In [3]:
options = Options()
options.add_argument("--headless")

driver = webdriver.Chrome(options=options)

In [4]:
categories = [
    {"name": "toys-and-games", "url": "https://www.amazon.com/Best-Sellers-Toys-Games/zgbs/toys-and-games/"},
    {"name": "electronics", "url": "https://www.amazon.com/Best-Sellers-Electronics/zgbs/electronics/"},
    {"name": "fashion", "url": "https://www.amazon.com/Best-Sellers/zgbs/fashion/"}
]

In [5]:
regex_options = [
    r"https://www.amazon.com/gp/product/(?P<product_id>[\w-]+)/",
    r"https://www.amazon.com/dp/(?P<product_id>[\w-]+)/",
    r"https://www.amazon.com/(?P<slug>[\w-]+)/dp/(?P<product_id>[\w-]+)/",
]

def extract_product_id_from_url(url):
    product_id = None
    for regex_str in regex_options:
        regex = re.compile(regex_str)
        match = regex.match(url)
        if match != None:
            try:
                product_id = match['product_id']
            except:
                pass
    return product_id

In [6]:
def clean_page_links(page_links=[], category=None):
    final_page_links = []
    for url in page_links:
        product_id = extract_product_id_from_url(url)
        if product_id != None:
            final_page_links.append({"url": url, "product_id": product_id, "category": category})
    return final_page_links

In [7]:
def scrace_category_product_links(categories=[]):
    all_product_links = []
    for category in categories:
        time.sleep(1.5)
        url = category.get("url")
        driver.get(url)
        body_el = driver.find_element_by_css_selector("body")
        html_str = body_el.get_attribute("innerHTML")
        html_obj = HTML(html=html_str)
        page_links = [f"https://www.amazon.com{x}" for x in html_obj.links if x.startswith("/")]
        cleaned_links = clean_page_links(page_links=page_links, category=category)
        all_product_links += cleaned_links
    return all_product_links

In [8]:
def extract_categories_and_save(categories=[]):
    all_product_links = scrace_category_product_links(categories)
    category_df = pd.DataFrame(all_product_links)
    category_df.to_csv(product_category_links_output, index=False)

In [9]:
extract_categories_and_save(categories=categories)

In [10]:

def scrape_product_page(url, title_lookup = "#productTitle", price_lookup = "#priceblock_ourprice"):
    driver.get(url)
    time.sleep(1.5)
    body_el = driver.find_element_by_css_selector("body")
    html_str = body_el.get_attribute("innerHTML")
    html_obj = HTML(html=html_str)
    product_title = html_obj.find(title_lookup, first=True).text
    product_price = html_obj.find(price_lookup, first=True).text
    return product_title, product_price

In [11]:
def perform_scrape(cleaned_items=[]):
    data_extracted = []
    for obj in cleaned_items:
        link = obj['url']
        product_id = obj['product_id']
        title, price = (None, None)
        try:
            title, price = scrape_product_page(link)
        except:
            pass
        if title != None and price != None:
            print(link, title, price)
        product_data = {
            "url": link,
            "product_id": product_id,
            "title": title,
            "price": price
        }
        data_extracted.append(product_data)
    return data_extracted

In [12]:
def row_scrape_event(row, *args, **kwargs):
    link = row['url']
    scraped = 0
    try:
        scraped = row['scraped']
    except:
        pass
    # print(link)
    if scraped == 1 or scraped == "1":
        print("skipped")
        return row
    product_id = row['product_id']
    title, price = (None, None)
    try:
        title, price = scrape_product_page(link)
    except:
        pass
    row['title'] = title
    row['price'] = price
    row['scraped'] = 1
    row['timestamp'] = datetime.datetime.now().timestamp()
    print(link, title, price)
    return row

In [13]:
df = pd.read_csv(product_category_links_output)
df.head()

,category,product_id,url
0,"{'name': 'toys-and-games', 'url': 'https://www...",B07ZFHR7PX,https://www.amazon.com/Villana-Dinosaur-Includ...
1,"{'name': 'toys-and-games', 'url': 'https://www...",B00062J99K,https://www.amazon.com/Crayola-Twistables-Clas...
2,"{'name': 'toys-and-games', 'url': 'https://www...",B00NHQF65S,https://www.amazon.com/LEGO-Classic-Green-Base...
3,"{'name': 'toys-and-games', 'url': 'https://www...",B01C5A2WJO,https://www.amazon.com/Little-Tikes-EasyScore-...
4,"{'name': 'toys-and-games', 'url': 'https://www...",B074RCV4HQ,https://www.amazon.com/Mattel-Games-FPD61-Mine...


In [14]:
df.shape

(149, 3)

In [15]:
df_sub = df.copy()

In [ ]:
df_sub = df_sub.apply(row_scrape_event, axis=1)

https://www.amazon.com/Villana-Dinosaur-Including-Triceratops-Christmas/dp/B07ZFHR7PX/ref=zg_bs_toys-and-games_35/145-8865603-7995100?_encoding=UTF8&psc=1&refRID=BBAFJKNA3N3RBX2X8TKC None None
https://www.amazon.com/Villana-Dinosaur-Including-Triceratops-Christmas/dp/B07ZFHR7PX/ref=zg_bs_toys-and-games_35/145-8865603-7995100?_encoding=UTF8&psc=1&refRID=BBAFJKNA3N3RBX2X8TKC None None
https://www.amazon.com/Crayola-Twistables-Classrooms-Preschools-Self-Sharpening/dp/B00062J99K/ref=zg_bs_toys-and-games_31/145-8865603-7995100?_encoding=UTF8&psc=1&refRID=BBAFJKNA3N3RBX2X8TKC Crayola Twistables Crayons Coloring Set, Kids Indoor Activities at Home, 24 Count $4.49
https://www.amazon.com/LEGO-Classic-Green-Baseplate-Supplement/dp/B00NHQF65S/ref=zg_bs_toys-and-games_34/145-8865603-7995100?_encoding=UTF8&psc=1&refRID=BBAFJKNA3N3RBX2X8TKC None None
https://www.amazon.com/Little-Tikes-EasyScore-Basketball-Exclusive/dp/B01C5A2WJO/ref=zg_bs_toys-and-games_22/145-8865603-7995100?_encoding=UTF8&psc=1&r

https://www.amazon.com/LEGO-Classic-Medium-Creative-Brick/dp/B00NHQFA1I/ref=zg_bs_toys-and-games_14/145-8865603-7995100?_encoding=UTF8&psc=1&refRID=BBAFJKNA3N3RBX2X8TKC None None
https://www.amazon.com/Crayola-Count-Sidewalk-Chalk-51-2048/dp/B00AHAJGXK/ref=zg_bs_toys-and-games_7/145-8865603-7995100?_encoding=UTF8&psc=1&refRID=BBAFJKNA3N3RBX2X8TKC None None
https://www.amazon.com/Play-Doh-B3423AS0-Confetti-Compound-Collection/dp/B011MIUQHA/ref=zg_bs_toys-and-games_18/145-8865603-7995100?_encoding=UTF8&psc=1&refRID=BBAFJKNA3N3RBX2X8TKC None None
https://www.amazon.com/Unicorn-Supplies-Everything-Glitter-Instructions/dp/B07C2YPYBM/ref=zg_bs_toys-and-games_41/145-8865603-7995100?_encoding=UTF8&psc=1&refRID=BBAFJKNA3N3RBX2X8TKC Original Stationery Unicorn Slime Kit Supplies Stuff for Girls Making Slime [Everything in One Box] Kids Can Make Unicorn, Glitter, Fluffy Cloud, Floam Putty, Pink $29.99
https://www.amazon.com/Disney-Princess-Necklace-Activity-Set/dp/B0747ZGTTG/ref=zg_bs_toys-and-ga

https://www.amazon.com/Echo-Show-8/dp/B07RQ3TCT2/ref=zg_bs_electronics_32?_encoding=UTF8&psc=1&refRID=VX8QHHB1QYVFEHRY1GSD Echo Show 8 - HD 8" smart display with Alexa - Sandstone $79.99
https://www.amazon.com/Fire-Tablet-7/dp/B07HZHJGY7/ref=zg_bs_electronics_26?_encoding=UTF8&psc=1&refRID=VX8QHHB1QYVFEHRY1GSD Fire 7 Tablet (7" display, 16 GB) - Twilight Blue $49.99
https://www.amazon.com/Roku-Premiere-HDR-Streaming-Player-Premium/dp/B07HDBZN7Q/ref=zg_bs_electronics_4?_encoding=UTF8&psc=1&refRID=VX8QHHB1QYVFEHRY1GSD Roku Premiere | HD/4K/HDR Streaming Media Player, Simple Remote and Premium HDMI Cable $29.99
https://www.amazon.com/Roku-Streaming-Player-Premium-Headphones/dp/B07WVF9SL5/ref=zg_bs_electronics_38?_encoding=UTF8&psc=1&refRID=VX8QHHB1QYVFEHRY1GSD Roku Ultra | Streaming Media Player 4K/HD/HDR with Premium JBL Headphones 2019 $84.49
https://www.amazon.com/Fitbit-Fitness-Smartwatch-Tracking-Included/dp/B07TWFVDWT/ref=zg_bs_electronics_11?_encoding=UTF8&psc=1&refRID=VX8QHHB1QYVF

In [ ]:
products_df = pd.read_csv(product_output)

In [ ]:
final_df = pd.concat([products_df, df_sub])
final_df.to_csv(product_output, index=False)

In [ ]:
final_df.head()